In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext
import time


/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
list_city_dt = []

for file_in in os.listdir(r'../../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

#remove berlin
val_rmv = []
for val in list_city_dt:
    if val.find('berlin') > -1:
        val_rmv.append(val)
for val in val_rmv:
    list_city_dt.remove(val)

print(list_city_dt)
print(len(list_city_dt))


['frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00', 'lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00', 'paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']
12


In [4]:

# list_workforce_xlsx = []

# for base_file in list_city_dt:

#     base_file_xlsx = f'{base_file}_all_trials.xlsx'
#     if base_file_xlsx in os.listdir(r'../../rerun_workforce_size'):
#         df_ = pd.read_excel(f'../../rerun_workforce_size/{base_file_xlsx}')
#         list_workforce_xlsx.append(df_)



list_workforce_xlsx = []

for base_file in list_city_dt:

    base_file_xlsx = f'{base_file}.json'
    if base_file_xlsx in os.listdir(r'../../workforce_size'):
        with open(f'../../workforce_size/{base_file}.json', 'r') as f:
            dict_data = json.load(f)
        df_ = pd.DataFrame(dict_data)
        list_workforce_xlsx.append(df_)



In [12]:
df_workforce = pd.concat(list_workforce_xlsx, ignore_index = True)

# df_workforce.columns = ['instance', 'city', 'DB', 'OC',
#        'RM', 'GM', 'model', 'max_n_shifts',
#        'workforce_size', 'wage_costs', 'objective_value',
#        'objective_value_post_wage', 'objval_round','workforce_size_trial','run_time','dict_run']
# df_workforce.loc[df_workforce['max_n_shifts'].isna(), 'max_n_shifts'] = 0
# df_workforce.drop_duplicates(subset = ['instance','OC','model','max_n_shifts','workforce_size_trial'], inplace = True)
# df_workforce['max_'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['workforce_size_trial'].transform('max')
# df_workforce = df_workforce[df_workforce['max_']!=df_workforce['workforce_size_trial']]

# df_workforce['max_'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['workforce_size_trial'].transform('max')
# df_workforce = df_workforce[df_workforce['max_']==df_workforce['workforce_size_trial']]
# df_workforce.drop(columns = ['max_'], inplace = True)

df_workforce['workforce_size_trial'] = df_workforce['workforce_size_region0']
df_workforce.rename(columns = {'outsourcing_cost_multiplier':'OC','instance_file_base':'instance'}, inplace = True)
df_workforce.loc[df_workforce['max_n_shifts'].isna(), 'max_n_shifts'] = 0

df_workforce.head(2)
print(df_workforce.columns)
print(len(df_workforce['OC']))

Index(['instance', 'city', 'demand_baseline', 'model', 'max_n_shifts', 'OC',
       'regional_multiplier', 'global_multiplier', 'workforce_size_region0',
       'objective_value_prev', 'objective_value', 'objective_value_next',
       'workforce_size_trial'],
      dtype='object')
300


In [14]:
df_workforce.sort_values(by = ['instance','model','max_n_shifts','OC'], inplace = True)
df_workforce.reset_index(drop = True, inplace = True)

df_workforce_ = df_workforce.copy()

df_workforce_['lag1'] = df_workforce_.groupby(by = ['instance','model','max_n_shifts'])['workforce_size_trial'].shift(1)
df_workforce_ = df_workforce_[df_workforce_['lag1'].notna()]

df_workforce_ = df_workforce_[['instance','model','max_n_shifts','OC','workforce_size_trial','lag1']]

df_workforce_['cndn'] = 0
df_workforce_.loc[(df_workforce_['lag1']<df_workforce_['workforce_size_trial']),'cndn'] = 1

# df_workforce_['cndn'] = 0
# df_workforce_.loc[df_workforce_['lag1']==(df_workforce_['workforce_size_trial']),'cndn'] = 1

# df_workforce_['cndn'] = 0
# df_workforce_.loc[(df_workforce_['lag1']>df_workforce_['workforce_size_trial']),'cndn'] = 1

print(df_workforce_['cndn'].sum())
print(len(df_workforce_['instance']))
df_workforce_.head(20)

df_check = df_workforce_[df_workforce_['cndn']==0].copy()
df_check

162
240


,instance,model,max_n_shifts,OC,workforce_size_trial,lag1,cndn
3,frankfurt_db=0.50,fixed,0.0,2.0,20,20.0,0
8,frankfurt_db=0.50,flex,0.0,2.0,17,17.0,0
13,frankfurt_db=0.50,partflex,2.0,2.0,20,20.0,0
18,frankfurt_db=0.50,partflex,3.0,2.0,17,17.0,0
23,frankfurt_db=0.50,partflex,4.0,2.0,17,17.0,0
...,...,...,...,...,...,...,...
246,paris_db=1.00,partflex,4.0,1.5,52,52.0,0
249,paris_db=1.00,partflex,4.0,2.5,55,55.0,0
258,paris_db=2.00,flex,0.0,2.0,99,99.0,0
268,paris_db=2.00,partflex,3.0,2.0,99,99.0,0


In [5]:
df_workforce['count'] = 1
df_workforce['count'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['count'].transform('sum')

df_workforce_count = df_workforce.copy()
df_workforce_count.drop_duplicates(subset = ['instance','OC','model','max_n_shifts'], inplace = True)

df_workforce_count.sort_values(by = ['count'], inplace = True)

In [8]:
df_workforce_count_ = df_workforce_count[df_workforce_count['count']==3].copy()
df_workforce_count_

,instance,city,DB,OC,RM,GM,model,max_n_shifts,workforce_size,wage_costs,objective_value,objective_value_post_wage,objval_round,workforce_size_trial,run_time,dict_run,count
1278,lyon_db=4.00,lyon,4.0,2.5,1.5,0.8,flex,0.0,82,1968.0,47995.571136,46027.571136,47995.57,79,50.494599,"{'fixed-None': ['partflex-2'], 'flex-None': ['...",3
1279,lyon_db=4.00,lyon,4.0,2.5,1.5,0.8,partflex,3.0,82,1968.0,47995.571136,46027.571136,47995.57,79,50.494599,"{'fixed-None': ['partflex-2'], 'flex-None': ['...",3
1280,lyon_db=4.00,lyon,4.0,2.5,1.5,0.8,partflex,4.0,82,1968.0,47995.571136,46027.571136,47995.57,79,50.494599,"{'fixed-None': ['partflex-2'], 'flex-None': ['...",3


In [3]:
df_ = pd.read_excel(f'../code_cm/df_shifts_example.xlsx')
df_.sort_values(by = ['city','demand_baseline','demand_type','outsourcing_cost_multiplier','model','max_n_shifts','region'], inplace = True)

df_.to_excel(f'../code_cm/df_shifts_example_sorted.xlsx', index=False)